In [1]:
import tensorflow as tf

<pre>
softmax function:
    n개를 예측할 때 유용

In [2]:
tf.set_random_seed(777) #for reproducibility

In [3]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

In [9]:
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 3])
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# softmax

<pre>
tf.nn.softmax -> softmax activation
softmax = exp(logits) / reduce_sum(exp(logits), dim)

In [10]:
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

## Testing one-hot encoding

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))
            
    print('----------------')
    
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a, 1)))
    
    print('----------------')
    
    b = sess.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
    print(b, sess.run(tf.argmax(b, 1)))
    
    print('----------------')
    all = sess.run(hypothesis, feed_dict = {X: [[1, 11, 7, 9], [1, 3, 4, 3]]})
    print(all, sess.run(tf.argmax(all, 1)))

0 2.5326858
200 0.9242872
400 0.770864
600 0.6855532
800 0.62745416
1000 0.58664834
1200 0.5565878
1400 0.53316146
1600 0.513999
1800 0.49773782
2000 0.483559
----------------
[[9.6372253e-01 3.6109433e-02 1.6808916e-04]] [0]
----------------
[[0.13032083 0.6301671  0.23951207]] [1]
----------------
[[9.6372253e-01 3.6109433e-02 1.6808916e-04]
 [1.3032083e-01 6.3016713e-01 2.3951207e-01]] [0 1]


# lab06_02

In [31]:
import numpy as np
import os
os.chdir("C:/Users/snuist/Desktop/Git/study/deeplearning")

In [44]:
# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

(101, 16) (101, 1)


In [33]:
nb_classes = 7

In [45]:
X = tf.placeholder(tf.float32, shape=[None, 16])
Y = tf.placeholder(tf.int32, shape=[None, 1])

In [46]:
#need to reshape of one_hot_encoding
Y_one_hot = tf.one_hot(Y, nb_classes)
print("one_hot", Y_one_hot, "shape", Y_one_hot.shape)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape", Y_one_hot)

one_hot Tensor("one_hot_4:0", shape=(?, 1, 7), dtype=float32) shape (?, 1, 7)
reshape Tensor("Reshape_2:0", shape=(?, 7), dtype=float32)


In [47]:
W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='biase')

## hypothesis

In [48]:
logits = tf.matmul(X, W) + b #logit or score
hypothesis = tf.nn.softmax(logits) #probability

## loss function

In [49]:
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)

<pre>
- Logistic Regression(Y= 0 or 1) cost:
   cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1-Y)*(1 - hypothesis))

- Softmax Classification(Y= 0~6) cost:
    cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
    
- Fancy Softmax Classification cost:
    cost_ i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
    cost = tf.reduce_mean(cost_i)

## Gradient descent

In [50]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

## Accuracy

In [51]:
prediction = tf.argmax(hypothesis, 1) # Y_
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1)) #Y_ vs Y
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Train the model

In [52]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={
                                 X: x_data, Y: y_data})
            print("Step: {:5} \tLoss: {:.3f} \tAcc: {:.2%}".format(
                step, loss, acc))

Step:     0 	Loss: 5.704 	Acc: 25.74%
Step:   100 	Loss: 0.740 	Acc: 77.23%
Step:   200 	Loss: 0.489 	Acc: 88.12%
Step:   300 	Loss: 0.382 	Acc: 90.10%
Step:   400 	Loss: 0.316 	Acc: 91.09%
Step:   500 	Loss: 0.269 	Acc: 94.06%
Step:   600 	Loss: 0.233 	Acc: 95.05%
Step:   700 	Loss: 0.205 	Acc: 95.05%
Step:   800 	Loss: 0.182 	Acc: 95.05%
Step:   900 	Loss: 0.163 	Acc: 95.05%
Step:  1000 	Loss: 0.147 	Acc: 96.04%
Step:  1100 	Loss: 0.133 	Acc: 96.04%
Step:  1200 	Loss: 0.121 	Acc: 96.04%
Step:  1300 	Loss: 0.111 	Acc: 98.02%
Step:  1400 	Loss: 0.103 	Acc: 98.02%
Step:  1500 	Loss: 0.095 	Acc: 98.02%
Step:  1600 	Loss: 0.088 	Acc: 98.02%
Step:  1700 	Loss: 0.082 	Acc: 99.01%
Step:  1800 	Loss: 0.077 	Acc: 99.01%
Step:  1900 	Loss: 0.072 	Acc: 99.01%
